In [17]:
import os
import time
import random
import optuna
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

import cv2
import json
import gc 
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt


# 시드 고정
def set_seed(SEED=42):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True

train_img_path = "../data/train_enhanced/"
train_csv_path = "../data/train2.csv"

test_img_path = "../data/test_enhanced/"
sample_path = "../data/sample_submission.csv"



# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, data, path, transform=None):
        """
        Args:
            data: DataFrame 또는 CSV 파일 경로
            path: 이미지 파일들이 있는 디렉토리 경로
            transform: 이미지 변환 함수
        """
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values
            
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        
        # RGB가 아닌 이미지 처리
        if len(img.shape) == 2:
            img = np.stack([img] * 3, axis=-1)
        elif img.shape[2] == 4:
            img = img[:, :, :3]
            
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

def get_transforms(img_size):
    trn_transform = A.Compose([
        # A.OneOf([
        # A.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=True, p=1),
        # A.ElasticTransform(always_apply=True, p=1, alpha=1.0, sigma=50.0, alpha_affine=50.0, interpolation=0, border_mode=1, value=(0, 0, 0), mask_value=None, approximate=False),
        # A.OpticalDistortion(always_apply=True, p=1, distort_limit=(-0.3, -0.1)),
        # A.OpticalDistortion(always_apply=True, p=1, distort_limit=(0.1, 0.3)),
        # ], p=0.85),
        # A.SomeOf([
        #     A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1),
        #     A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, val_shift_limit=20, p=1),
        #     A.MultiplicativeNoise(p=1, multiplier=(1, 1.5), per_channel=True),
        #     A.Equalize(p=1, mode='cv', by_channels=True),
        # ], n=2, p=0.85),
        A.OneOf([
            A.Rotate(limit=(10, 30), border_mode=cv2.BORDER_CONSTANT, p=1),
            A.Rotate(limit=(150, 170), border_mode=cv2.BORDER_CONSTANT, p=1),
            A.Rotate(limit=(190, 210), border_mode=cv2.BORDER_CONSTANT, p=1),
            A.Rotate(limit=(330, 350), border_mode=cv2.BORDER_CONSTANT, p=1),
        ], p=1),
        # A.CoarseDropout(p=0.5, max_holes=40, max_height=15, max_width=15, min_holes=8, min_height=8, min_width=8),
        # A.Equalize(p=0.5, mode='cv', by_channels=True),
        # A.OneOf([
        #     A.Blur(blur_limit=(3, 4), p=1),
        #     A.MotionBlur(blur_limit=(3, 5), p=1),
        #     A.Downscale(scale_min=0.455, scale_max=0.5, interpolation=2, p=1),
        # ], p=0.5),
        # A.GaussNoise(var_limit=(100, 800), per_channel=True, p=0.5),
        A.RandomRotate90(p=0.4),
        A.HorizontalFlip(p=0.4), 
        A.VerticalFlip(p=0.4),
        # A.CLAHE(p=0.5),
        # A.MotionBlur(p=0.2),
        # A.RandomBrightnessContrast(p=0.2),
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    tst_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    
    return trn_transform, tst_transform


def load_multiple_datasets(csv_paths, img_paths, transform=None):
    """
    여러 CSV 파일과 이미지 폴더들을 하나의 데이터셋으로 통합
    
    Args:
        csv_paths (list): CSV 파일 경로들의 리스트
        img_paths (list): 이미지 폴더 경로들의 리스트
        transform: 이미지 변환을 위한 transform 함수
    """
    if len(csv_paths) != len(img_paths):
        raise ValueError("CSV 파일 개수와 이미지 폴더 개수가 일치해야 합니다.")
    
    # 각 소스별로 데이터셋 생성
    datasets = []
    for csv_path, img_path in zip(csv_paths, img_paths):
        dataset = ImageDataset(csv_path, img_path, transform=transform)
        datasets.append(dataset)
    
    # 모든 데이터셋 통합
    combined_dataset = ConcatDataset(datasets)
    return combined_dataset






def calculate_class_metrics(y_true, y_pred, num_classes=17):
    """각 클래스별 accuracy와 f1 score를 계산"""
    class_metrics = {}
    
    for class_idx in range(num_classes):
        # 해당 클래스에 대한 이진 레이블 생성
        y_true_binary = (y_true == class_idx)
        y_pred_binary = (y_pred == class_idx)
        
        # 클래스별 metrics 계산
        class_acc = accuracy_score(y_true_binary, y_pred_binary)
        class_f1 = f1_score(y_true_binary, y_pred_binary, average='binary')
        support = np.sum(y_true_binary)
        
        class_metrics[f'class_{class_idx}'] = {
            'accuracy': class_acc,
            'f1_score': class_f1,
            'support': support
        }
        
    # 전체 metrics
    overall_acc = accuracy_score(y_true, y_pred)
    overall_f1 = f1_score(y_true, y_pred, average='macro')
    
    class_metrics['overall'] = {
        'accuracy': overall_acc,
        'f1_score': overall_f1
    }
    
    return class_metrics    
        
        
        
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader, leave=False)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)
        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return {
        "loss": train_loss,
        "acc": train_acc,
        "f1": train_f1,
    }

def valid_one_epoch(loader, model, loss_fn, device):
    model.eval()
    valid_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            valid_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    valid_loss /= len(loader)
    
    # Confusion Matrix 계산
    conf_matrix = confusion_matrix(targets_list, preds_list)
    metrics = calculate_class_metrics(np.array(targets_list), np.array(preds_list))

    return {
        "loss": valid_loss,
        "metrics": metrics,
        "confusion_matrix": conf_matrix  # confusion matrix 추가
    }




def evaluate_best_params(params, device):
    """베스트 파라미터로 k-fold 검증 수행"""
    print("\nEvaluating best parameters with k-fold validation...")
    
    if params['model_name'] == "swinv2_tiny_window8_256":
        img_size = 256
    elif params['model_name'] == "tf_efficientnet_b3.ns_jft_in1k":
        img_size = 300
    elif params['model_name'] == "eva02_base_patch14_448.mim_in22k_ft_in22k_in1k":
        img_size = 448
        
    trn_transform, _ = get_transforms(img_size)
    
    # -----------------------------------------------------------------------------
    # K-fold 검증 준비
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # 전체 데이터셋 로드
    full_dataset = ImageDataset(train_csv_path, train_img_path, transform=trn_transform)
    
    # 데이터와 라벨 분리
    data = pd.read_csv(train_csv_path)
    X = np.arange(len(data))
    y = data['target'].values
    
    # 각 fold의 결과를 저장할 리스트
    fold_results = []
    all_confusion_matrices = []  # confusion matrix 저장용 리스트 추가
    
    
    
    # K-fold 교차 검증
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"\n{'='*20} Fold {fold} {'='*20}")
        
        # 데이터셋 분할
        train_dataset = Subset(full_dataset, train_idx)
        val_dataset = Subset(full_dataset, val_idx)
        
        train_loader = DataLoader(
            train_dataset,
            batch_size=params['batch_size'],
            shuffle=True,
            num_workers=4,
            pin_memory=True
        )
        
        val_loader = DataLoader(
            val_dataset,
            batch_size=params['batch_size'],
            shuffle=False,
            num_workers=4,
            pin_memory=True
        )
        
        # 모델 설정
        model = timm.create_model(
            params['model_name'],
            pretrained=True,
            num_classes=17
        ).to(device)
        
        optimizer_class = getattr(torch.optim, params['optimizer'])
        optimizer = optimizer_class(model.parameters(), 
                                  lr=params['lr'], 
                                  weight_decay=params['weight_decay'])
        
        loss_fn = nn.CrossEntropyLoss()
        
        # -----------------------------------------------------------------------------
        best_val_f1 = 0
        best_epoch_metrics = None
        patience = 5
        patience_counter = 0
        EPOCH = 3
        
        # 학습
        for epoch in range(EPOCH):
            train_ret = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
            val_ret = valid_one_epoch(val_loader, model, loss_fn, device)
            
            current_val_f1 = val_ret['metrics']['overall']['f1_score']
            
            if current_val_f1 > best_val_f1:
                best_val_f1 = current_val_f1
                best_epoch_metrics = val_ret['metrics']
                best_confusion_matrix = val_ret['confusion_matrix']  # 최고 성능 모델의 confusion matrix 저장
                patience_counter = 0
            else:
                patience_counter += 1
            
            if not epoch % 5:
                print(f"Epoch {epoch+1}: Val F1 = {current_val_f1:.4f}")
            
            if patience_counter >= patience:
                print("Early stopping!")
                break
            
                    
        
        # 현재 fold의 best 결과 출력
        # print(f"\nBest Results for Fold {fold}:")
        # print("\nClass-wise Metrics:")
        # for class_idx in range(17):
        #     metrics = best_epoch_metrics[f'class_{class_idx}']
        #     print(f"\nClass {class_idx}:")
        #     print(f"Accuracy: {metrics['accuracy']:.4f}")
        #     print(f"F1 Score: {metrics['f1_score']:.4f}")
        #     print(f"Support: {metrics['support']}")
        
        # 현재 fold의 confusion matrix 출력
        # print(f"\nConfusion Matrix for Fold {fold}:")
        # print(best_confusion_matrix)
        
        # fold 결과 저장
        fold_results.append({
            'fold': fold,
            'metrics': best_epoch_metrics
        })
        
        all_confusion_matrices.append(best_confusion_matrix)
    
    # 모든 fold의 평균 성능 계산 및 출력
    print("\n" + "="*50)
    print("Average Performance Across All Folds:")

    
    
    # 클래스별 평균 성능
    class_avg_metrics = {}
    # for class_idx in range(17):
    #     accuracies = [fold['metrics'][f'class_{class_idx}']['accuracy'] for fold in fold_results]
    #     f1_scores = [fold['metrics'][f'class_{class_idx}']['f1_score'] for fold in fold_results]
        
    #     avg_acc = np.mean(accuracies)
    #     avg_f1 = np.mean(f1_scores)
    #     std_acc = np.std(accuracies)
    #     std_f1 = np.std(f1_scores)
        
    #     print(f"\nClass {class_idx}:")
    #     print(f"Accuracy: {avg_acc:.4f} (±{std_acc:.4f})")
    #     print(f"F1 Score: {avg_f1:.4f} (±{std_f1:.4f})")
    
    # 결과를 DataFrame으로 저장
    results_dict = {
        'fold': [],
        'class': [],
        'accuracy': [],
        'f1_score': [],
        'support': []
    }
    
    for fold_result in fold_results:
        fold_num = fold_result['fold']
        metrics = fold_result['metrics']
        
        for class_idx in range(17):
            class_metrics = metrics[f'class_{class_idx}']
            results_dict['fold'].append(fold_num)
            results_dict['class'].append(class_idx)
            results_dict['accuracy'].append(class_metrics['accuracy'])
            results_dict['f1_score'].append(class_metrics['f1_score'])
            results_dict['support'].append(class_metrics['support'])
    
    results_df = pd.DataFrame(results_dict)
    now = datetime.now()
    results_df.to_csv(f'best_params_fold_class_metrics_{now.month:02}{now.day:02}{now.hour:02}{now.minute:02}.csv', index=False)
    print("\nDetailed metrics saved to 'best_params_fold_class_metrics.csv'")
    
    
    
    # 모든 fold의 confusion matrix 평균 계산
    avg_confusion_matrix = np.mean(all_confusion_matrices, axis=0)
    std_confusion_matrix = np.std(all_confusion_matrices, axis=0)
    
    # 평균 Confusion Matrix 시각화
    plt.figure(figsize=(15, 6))
    
    # 평균 Confusion Matrix
    plt.subplot(1, 2, 1)
    sns.heatmap(avg_confusion_matrix, annot=True, fmt='.1f', cmap='Blues')
    plt.title('Average Confusion Matrix Across All Folds')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # 표준편차 Confusion Matrix
    plt.subplot(1, 2, 2)
    sns.heatmap(std_confusion_matrix, annot=True, fmt='.2f', cmap='Reds')
    plt.title('Standard Deviation of Confusion Matrix Across All Folds')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # 저장
    plt.savefig(f'confusion_matrix_average_{now.month:02}{now.day:02}{now.hour:02}{now.minute:02}.png')
    plt.close()
    
    # print("\nAverage Confusion Matrix Across All Folds:")
    # print(avg_confusion_matrix)
    # print("\nStandard Deviation of Confusion Matrix Across All Folds:")
    # print(std_confusion_matrix)
    
    return fold_results



def generate_pseudo_labels(model, loader, device):
    model.eval()
    pseudo_labels = []
    confidences = []
    
    with torch.no_grad():
        for image, _ in tqdm(loader, desc="Generating pseudo labels"):
            image = image.to(device)
            outputs = model(image)
            probs = torch.softmax(outputs, dim=1)
            conf, preds = torch.max(probs, dim=1)
            pseudo_labels.extend(preds.cpu().numpy())
            confidences.extend(conf.cpu().numpy())
            
    return np.array(pseudo_labels), np.array(confidences)

def objective(trial , device):
    try:
        
        print(f"GPU Memory before trial: {torch.cuda.memory_allocated(device)/(1024**2):.2f}MB")
        # 하이퍼파라미터 탐색 공간 정의
        params = {
            # 'model_name': trial.suggest_categorical('model_name', ['swinv2_tiny_window8_256', 'tf_efficientnet_b3.ns_jft_in1k', 'resnet18', 'eva02_base_patch14_448.mim_in22k_ft_in22k_in1k']),
            'model_name': trial.suggest_categorical('model_name', ['eva02_base_patch14_448.mim_in22k_ft_in22k_in1k']),
            # 'img_size': trial.suggest_categorical('img_size', [256]),
            'batch_size': trial.suggest_categorical('batch_size', [16]),
            'lr': trial.suggest_float('lr', 1e-4, 1e-3, log=True),
            'weight_decay' : trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True),
            'dropout_rate' : trial.suggest_float('dropout_rate', 0.0, 0.3),
            'optimizer': trial.suggest_categorical('optimizer', ['Adam', 'AdamW']),
        }
    
        
        # 모델 선택 및 입력 이미지 크기 설정
        if params['model_name'] == "swinv2_tiny_window8_256":
            params['img_size'] = 256
        elif params['model_name'] == "tf_efficientnet_b3.ns_jft_in1k":
            params['img_size'] = 300
        elif params['model_name'] == "eva02_base_patch14_448.mim_in22k_ft_in22k_in1k":
            params['img_size'] = 448
            
        # 데이터셋 준비
        trn_transform, val_transform = get_transforms(params['img_size'])
        
        # -----------------------------------------------------------------------------
        # K-fold 검증 준비
        n_splits = 5
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        
        # 전체 데이터셋 로드
        full_dataset = ImageDataset(train_csv_path, train_img_path, transform=trn_transform)
        
        # 데이터와 라벨 분리
        data = pd.read_csv(train_csv_path)
        X = np.arange(len(data))
        y = data['target'].values
        
        fold_scores = []
        
        # K-fold 교차 검증
        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
            print(f"\nFold {fold}")
            
            # 데이터셋 분할
            train_dataset = Subset(full_dataset, train_idx)
            val_dataset = Subset(full_dataset, val_idx)
            
            train_loader = DataLoader(
                train_dataset,
                batch_size=params['batch_size'],
                shuffle=True,
                num_workers=4,
                pin_memory=True
            )
            
            val_loader = DataLoader(
                val_dataset,
                batch_size=params['batch_size'],
                shuffle=False,
                num_workers=4,
                pin_memory=True
            )
            
            # 모델 설정
            model = timm.create_model(
                params['model_name'],
                pretrained=True,
                num_classes=17
            ).to(device)
            
            # Optimizer 설정
            optimizer_class = getattr(torch.optim, params['optimizer'])
            optimizer = optimizer_class(model.parameters(), lr=params['lr'])
            
            loss_fn = nn.CrossEntropyLoss()
            
            # -----------------------------------------------------------------------------
            best_val_f1 = 0
            patience = 5
            patience_counter = 0
            EPOCH = 5
            
            # 학습
            for epoch in range(EPOCH):
                train_ret = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
                val_ret = valid_one_epoch(val_loader, model, loss_fn, device)
                current_val_f1 = val_ret['metrics']['overall']['f1_score']
                
                    
                if not epoch % 10:
                    print(f"Epoch {epoch+1}: Train F1 = {train_ret['f1']:.4f}, Val F1 = {current_val_f1:.4f}")
                
                if current_val_f1 > best_val_f1:
                    best_val_f1 = current_val_f1
                    patience_counter = 0
                else:
                    patience_counter += 1
                    
                if patience_counter >= patience:
                    print("Early stopping!")
                    break
                
                if current_val_f1 < 0.5:
                    print("Early stopping!")
                    break
         
            fold_scores.append(best_val_f1)
            
        print(f"GPU Memory after trial: {torch.cuda.memory_allocated(device)/(1024**2):.2f}MB")    
        
        mean_f1 = np.mean(fold_scores)
        return mean_f1
    
    finally:
        torch.cuda.empty_cache()
        gc.collect()
        
        
        
        

In [10]:

set_seed()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device=="cuda": torch.cuda.empty_cache()

In [4]:


# Optuna를 사용한 하이퍼파라미터 최적화
study = optuna.create_study(
    direction='maximize',
    storage=optuna.storages.JournalStorage(
        optuna.storages.JournalFileStorage("study.log")  # 로그 파일로 저장
    ),
    load_if_exists=True
)
study.optimize(lambda trial: objective(trial, device), n_trials=20)

best_params = study.best_params
print("Best parameters:", best_params)


# Best parameters 저장
now = datetime.now()
with open(f"best_params_{now.month:02}{now.day:02}{now.hour:02}{now.minute:02}.json", "w") as f:
    json.dump(best_params, f)
print("Best parameters saved to best_params.json")




/tmp/ipykernel_1416/1499754699.py:5: FutureWarning: JournalFileStorage has been deprecated in v4.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v4.0.0. Use :class:`~optuna.storages.journal.JournalFileBackend` instead.
  optuna.storages.JournalFileStorage("study.log")  # 로그 파일로 저장
[I 2024-11-06 06:24:07,136] A new study created in Journal with name: no-name-0b76a114-17df-45f9-9631-0782c62acc51


/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


GPU Memory before trial: 0.00MB

Fold 1


model.safetensors:   0%|          | 0.00/348M [00:00<?, ?B/s]

Epoch 1: Train F1 = 0.0738, Val F1 = 0.0201
Early stopping!

Fold 2


...

Epoch 1: Train F1 = 0.3339, Val F1 = 0.4991
Early stopping!

Fold 3


Epoch 1: Train F1 = 0.3090, Val F1 = 0.3078
Early stopping!

Fold 4


Epoch 1: Train F1 = 0.3259, Val F1 = 0.3370
Early stopping!

Fold 5


[I 2024-11-06 08:49:10,273] Trial 19 finished with value: 0.33972333382459075 and parameters: {'model_name': 'eva02_base_patch14_448.mim_in22k_ft_in22k_in1k', 'batch_size': 16, 'lr': 0.00013626716388884984, 'weight_decay': 0.00023426017489523913, 'dropout_rate': 0.26748964303942263, 'optimizer': 'Adam'}. Best is trial 14 with value: 0.471243297405415.


Epoch 1: Train F1 = 0.3127, Val F1 = 0.3202
Early stopping!
GPU Memory after trial: 1342.62MB
Best parameters: {'model_name': 'eva02_base_patch14_448.mim_in22k_ft_in22k_in1k', 'batch_size': 16, 'lr': 0.00013177562817053535, 'weight_decay': 0.00019072756574896346, 'dropout_rate': 0.21778562090390602, 'optimizer': 'Adam'}
Best parameters saved to best_params.json


In [18]:
torch.cuda.empty_cache()

In [25]:
# Best parameters 불러오기
with open("best_params_11050709.json", "r") as f:
    loaded_params = json.load(f)
print("Loaded best parameters:", loaded_params)


Loaded best parameters: {'model_name': 'tf_efficientnet_b3.ns_jft_in1k', 'batch_size': 16, 'lr': 0.00023815462688141223, 'weight_decay': 0.001496609249163004, 'dropout_rate': 0.12519174608631073, 'optimizer': 'Adam'}


In [48]:
# 베스트 파라미터로 k-fold 검증 수행
# loaded_params, best_params
fold_results = evaluate_best_params(loaded_params, device)


Evaluating best parameters with k-fold validation...

==================== Fold 1 ====================


Epoch 1: Val F1 = 0.6754



==================== Fold 2 ====================


Epoch 1: Val F1 = 0.6660



==================== Fold 3 ====================


...


==================== Fold 4 ====================


Epoch 1: Val F1 = 0.6667



==================== Fold 5 ====================


Epoch 1: Val F1 = 0.6545



Average Performance Across All Folds:

Detailed metrics saved to 'best_params_fold_class_metrics.csv'

Average Confusion Matrix Across All Folds:
[[20.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. ]
 [ 0.   7.2  0.   0.2  0.4  0.   0.   0.4  0.   0.   0.2  0.   0.   1.
   0.   0.   0. ]
 [ 0.   0.  19.8  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.2]
 [ 0.2  0.   0.   8.2  2.4  0.   0.   6.   0.   0.   0.2  0.   0.   0.2
   2.2  0.   0. ]
 [ 0.   0.8  0.   2.  10.8  0.   0.   2.   0.   0.   0.2  0.   0.   0.8
   3.2  0.   0. ]
 [ 0.   0.   0.   0.   0.  20.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.  19.   0.   0.   0.   0.4  0.4  0.   0.2
   0.   0.   0. ]
 [ 0.   0.   0.   5.6  4.   0.   0.2  7.4  0.   0.   0.2  0.   0.4  0.6
   1.8  0.   0. ]
 [ 0.   0.   0.   0.   0.2  0.4  0.   0.  19.6  0.   0.   0.   0.   0.
   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.  1

In [26]:

if loaded_params['model_name'] == "swinv2_tiny_window8_256":
    img_size = 256
elif loaded_params['model_name'] == "tf_efficientnet_b3.ns_jft_in1k":
    img_size = 300
elif loaded_params['model_name'] == "eva02_base_patch14_448.mim_in22k_ft_in22k_in1k":
    img_size = 448
            
# img_size = loaded_params['img_size']
trn_transform, tst_transform = get_transforms(img_size)

# 전체 학습 데이터셋으로 모델 학습
combined_csv_paths = [
    # train_csv_path,
    # '../data/train2.csv',
    '../data/train2_only_3_7.csv',
    '../data/target_3_images.csv',
    # '../data/target_4_images.csv',
    '../data/target_7_images.csv',
    # '../data/target_14_images.csv'
]

combined_img_paths = [
    # train_img_path,
    '../data/train_enhanced',
    '../data/cropped_images_enhanced/target_3',
    # '../data/cropped_images/target_4',
    '../data/cropped_images_enhanced/target_7',
    # '../data/cropped_images/target_14',
]

# 통합된 데이터셋 생성
combined_dataset = load_multiple_datasets(combined_csv_paths, combined_img_paths, transform=trn_transform)

# trn_dataset = ImageDataset(train_csv_path, train_img_path, transform=trn_transform)

trn_loader = DataLoader(
    combined_dataset,
    batch_size=loaded_params['batch_size'],
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

# 위에 명시되어 있음.  경로 바꿀 경우만
sample_path = '../data/output_only_3_7.csv'

tst_dataset = ImageDataset(sample_path, test_img_path, transform=tst_transform)

tst_loader = DataLoader(
    tst_dataset,
    batch_size=loaded_params['batch_size'],
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [27]:
# 최적의 하이퍼파라미터로 최종 모델 학습


# 최종 모델 학습
model = timm.create_model(
    loaded_params['model_name'],
    pretrained=True,
    num_classes=17
).to(device)

optimizer_class = getattr(torch.optim, loaded_params['optimizer'])
optimizer = optimizer_class(model.parameters(), lr=loaded_params['lr'])
loss_fn = nn.CrossEntropyLoss()


# -----------------------------------------------------------------------------
# 초기 학습
print("Training initial model...")
for epoch in range(100):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)
    # if epoch % 10 ==0:
    print(f"Epoch {epoch+1}: F1 = {ret['f1']:.4f}")


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

Training initial model...


Epoch 1: F1 = 0.0786


Epoch 2: F1 = 0.5959


Epoch 3: F1 = 0.6222


...

Epoch 96: F1 = 0.9798


Epoch 97: F1 = 0.9874


Epoch 98: F1 = 0.9874


Epoch 99: F1 = 0.9899


Epoch 100: F1 = 0.9823


In [ ]:

# Pseudo Labeling
print("\nGenerating pseudo labels...")
pseudo_labels, confidences = generate_pseudo_labels(model, tst_loader, device)

# 높은 신뢰도의 예측만 선택 (임계값: 0.9)
confidence_threshold = 0.9
high_confidence_mask = confidences > confidence_threshold

# Pseudo label 데이터 생성
test_data = pd.read_csv(sample_path)
pseudo_df = pd.DataFrame({
    'ID': test_data['ID'][high_confidence_mask],
    'target': pseudo_labels[high_confidence_mask]
})

# Pseudo label 데이터로 추가 학습
print(f"\nFound {len(pseudo_df)} high-confidence pseudo labels")
if len(pseudo_df) > 0:
    pseudo_dataset = ImageDataset(pseudo_df, test_img_path, transform=trn_transform)
    pseudo_loader = DataLoader(
        pseudo_dataset,
        batch_size=loaded_params['batch_size'],
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    
    # -----------------------------------------------------------------------------
    print("Training with pseudo labels...")
    for epoch in range(20):
        ret = train_one_epoch(pseudo_loader, model, optimizer, loss_fn, device)
        print(f"Pseudo Label Epoch {epoch+1}: F1 = {ret['f1']:.4f}")



In [14]:
def save_model(model, save_dir, model_name=None, additional_info=None):
    """
    모델을 저장하는 함수
    
    Parameters:
    model: 저장할 PyTorch 모델
    save_dir: 저장할 디렉토리 경로
    model_name: 모델 파일 이름 (None인 경우 타임스탬프 사용)
    additional_info: 모델과 함께 저장할 추가 정보 (옵션)
    """
    # 저장 디렉토리 생성
    os.makedirs(save_dir, exist_ok=True)
    
    # 파일명 생성
    if model_name is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        model_name = f'model_{timestamp}'
    
    # 저장할 데이터 준비
    save_dict = {
        'model_state_dict': model.state_dict(),
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    }
    
    # 추가 정보가 있는 경우 저장
    if additional_info:
        save_dict.update(additional_info)
    
    # 모델 저장
    save_path = os.path.join(save_dir, f'{model_name}.pth')
    torch.save(save_dict, save_path)
    
    print(f"Model saved to: {save_path}")
    return save_path

def load_model(model_name, model_path, num_classes = 17):
    """
    저장된 모델을 불러오는 함수
    
    Parameters:
    model: 불러올 모델 구조 (빈 모델)
    model_path: 저장된 모델 파일 경로
    
    Returns:
    model: 로드된 모델
    additional_info: 추가 정보 (있는 경우)
    """
    
    model = timm.create_model(
        model_name=model_name,
        pretrained=False,  # 저장된 가중치를 불러올 것이므로 False
        num_classes=num_classes
    ).to(device)
    
    # 모델 파일 로드
    checkpoint = torch.load(model_path)
    
    # 모델 가중치 로드
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # 추가 정보 반환
    additional_info = {k: v for k, v in checkpoint.items() if k != 'model_state_dict'}
    
    print(f"Model loaded from: {model_path}")
    print(f"Model timestamp: {additional_info.get('timestamp', 'N/A')}")
    
    return model, additional_info

In [15]:
save_model(
    model=model,
    save_dir='models/',
    model_name='eva02_base_patch14_448',
    # additional_info={
    #     'epoch': current_epoch,
    #     'train_loss': train_loss,
    #     'val_loss': val_loss,
    #     'accuracy': accuracy
    # }
)


Model saved to: models/eva02_base_patch14_448.pth


'models/eva02_base_patch14_448.pth'

In [29]:
# 3. 모델 불러오기
model, info = load_model(
    model_name='tf_efficientnet_b3.ns_jft_in1k',  # 빈 모델 구조
    model_path='models/tf_efficientnet_b3.ns_jft_in1k.pth'
)

Model loaded from: models/tf_efficientnet_b3.ns_jft_in1k.pth
Model timestamp: 2024-11-05 10:02:12


In [28]:
test_data = pd.read_csv(sample_path)

# 최종 예측 및 저장
print("\nGenerating final predictions...")
final_preds = []
model.eval()
with torch.no_grad():
    for image, _ in tqdm(tst_loader):
        image = image.to(device)
        preds = model(image)
        final_preds.extend(preds.argmax(dim=1).cpu().numpy())

pred_df = pd.DataFrame({
    'ID': test_data['ID'],
    'target': final_preds
})
now = datetime.now()
pred_df.to_csv(f"predictions_{now.month:02}{now.day:02}{now.hour:02}{now.minute:02}.csv", index=False)
print("\nPredictions saved to 'predictions.csv'")


Generating final predictions...


100%|██████████| 25/25 [00:01<00:00, 19.89it/s]


Predictions saved to 'predictions.csv'
